## Tutorial: Running Solution Optimization

![TextGrad](https://github.com/vinid/data/blob/master/logo_full.png?raw=true)

An autograd engine -- for textual gradients!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zou-group/TextGrad/blob/main/examples/notebooks/Prompt-Optimization.ipynb)
[![GitHub license](https://img.shields.io/badge/License-MIT-blue.svg)](https://lbesson.mit-license.org/)
[![Arxiv](https://img.shields.io/badge/arXiv-2406.07496-B31B1B.svg)](https://arxiv.org/abs/2406.07496)
[![Documentation Status](https://readthedocs.org/projects/textgrad/badge/?version=latest)](https://textgrad.readthedocs.io/en/latest/?badge=latest)
[![PyPI - Python Version](https://img.shields.io/pypi/pyversions/textgrad)](https://pypi.org/project/textgrad/)
[![PyPI](https://img.shields.io/pypi/v/textgrad)](https://pypi.org/project/textgrad/)

**Objectives:**

* In this tutorial, we will implement a solution optimization pipeline.

**Requirements:**

* You need to have an OpenAI API key to run this tutorial. This should be set as an environment variable as OPENAI_API_KEY.


In [2]:
!pip install textgrad # you might need to restart the notebook after installing textgrad

import os
from google.colab import userdata

# Set the OPENAI_API_KEY environment variable using Colab secrets
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

os.environ['OPENAI_API_KEY'] = 'sk-8heuJlDOeyAwZd65fRYl8x81sF3Jx5rSPIMsr8dFVwS7BmPu'

import textgrad as tg
tg.set_backward_engine(tg.get_engine("gpt-4o"))

initial_solution = """To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

solution = tg.Variable(initial_solution,
                       requires_grad=True,
                       role_description="solution to the math question")

loss_system_prompt = tg.Variable("""You will evaluate a solution to a math question.
Do not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise.""",
                                 requires_grad=False,
                                 role_description="system prompt")

loss_fn = tg.TextLoss(loss_system_prompt)
optimizer = tg.TGD([solution])

ValueError: Please set the OPENAI_API_KEY environment variable if you'd like to use OpenAI models.

In [ ]:
loss = loss_fn(solution)
loss

Variable(value=Errors:
1. Incorrect sign in the discriminant calculation: it should be b^2 - 4ac, not b^2 + 4ac.
2. Incorrect simplification of the quadratic formula: the denominator should be 2a, not 6.
3. Final solutions are missing the division by 2a., role=response from the language model, grads=)

In [ ]:
loss.backward()
optimizer.step()
print(solution.value)

To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a

Given:
a = 3, b = -7, c = 2

Substitute the values into the formula:
x = (7 ± √((-7)^2 - 4(3)(2))) / 6
x = (7 ± √(49 - 24)) / 6
x = (7 ± √25) / 6
x = (7 ± 5) / 6

The solutions are:
x1 = (7 + 5) / 6 = 12 / 6 = 2
x2 = (7 - 5) / 6 = 2 / 6 = 1/3
